In [ ]:
import pandas as pd
import numpy as np

import time
from datetime import datetime

from geopy.geocoders import MapBox

from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

## get city / prefecture geographic corrdinates

In [2]:
df = pd.read_csv('../Downloads/od_data_hokkaido/od_data_hokkaido.csv')

In [3]:
df['arr'] = df['Destination Prefecture']+df['Destination City']
df['dep'] = df['Origin Prefecture']+df['Origin City']

In [4]:
s_cityname = pd.concat([df['arr'], df['dep']]
                       ).drop_duplicates().reset_index(drop=True)
df_cityname = pd.DataFrame({'cityname': s_cityname})

In [ ]:
geolocator = MapBox(
    api_key='pk.eyJ1Ijoic3RpbGxiZW5nYmVuZyIsImEiOiJja2M1eHBpa2Qwamh2MnBxczhlcjlnazV0In0.hVW33t9_bUQ0lrtDOBl7aw')
for i in range(len(s_cityname)):
    if geolocator.geocode(s_cityname[i]) is None:
        print(s_cityname[i])
    else:
        df_cityname.loc[i, 'lat'] = geolocator.geocode(
            s_cityname[i], timeout=10).latitude
        df_cityname.loc[i, 'lon'] = geolocator.geocode(
            s_cityname[i], timeout=10).longitude

In [5]:
s_prefecturename = df['Origin Prefecture'].drop_duplicates(
).reset_index(drop=True)
df_prefecturename = pd.DataFrame({'prefecturename': s_prefecturename})

In [ ]:
geolocator = MapBox(
    api_key='pk.eyJ1Ijoic3RpbGxiZW5nYmVuZyIsImEiOiJja2M1eHBpa2Qwamh2MnBxczhlcjlnazV0In0.hVW33t9_bUQ0lrtDOBl7aw')
for i in range(len(s_prefecturename)):
    if geolocator.geocode(s_cityname[i]) is None:
        print(s_cityname[i])
    else:
        df_prefecturename.loc[i, 'lat'] = geolocator.geocode(
            s_prefecturename[i], timeout=10).latitude
        df_prefecturename.loc[i, 'lon'] = geolocator.geocode(
            s_prefecturename[i], timeout=10).longitude

In [6]:
df_cityname = pd.read_csv('../Documents/ipynb/Hokkaido_cityname.csv')

In [7]:
df_prefecturename = pd.read_csv(
    '../DDD_locationmind/Hokkaido_prefecturename.csv')

## data process

In [8]:

df_drop = df[~(df['arr'] == df['dep'])]
df_drop = df_drop.drop(['Arrcity', 'Depcity'], axis=1)

In [9]:
from datetime import datetime

df_drop['Date'] = df_drop['Date'].apply(
    lambda x: datetime.strptime(x, '%m/%d/%Y'))

In [10]:
import datetime

df_drop['time'] = df_drop['Date'].apply(lambda x: (
    x+datetime.timedelta(hours=1)).strftime('%m/%d/%Y %H:%M:%S'))

In [11]:
from datetime import datetime

df_drop['date'] = df_drop['Date'].apply(
    lambda x: '{:02d}/{:02d}'.format(x.month, x.day))
df_drop.loc['new_low', 'date'] = '02/29'

df_drop['year'] = df_drop['Date'].apply(lambda x: x.year)
df_drop['year'] = df_drop['year'].apply(lambda x: str(x))
df_drop.loc['new_low', 'year'] = '2019'

In [15]:
df_drop_19 = df_drop.loc[df_drop['year'] == '2019', :]
df_drop_20 = df_drop.loc[df_drop['year'] == '2020', :]

#### 2020 data overview

In [13]:
df_hkk_20 = df_drop_20[df_drop_20['Origin Prefecture'] == '北海道']
df_nothkk_20 = df_drop_20[df_drop_20['Origin Prefecture'] != '北海道']

In [14]:
df_hkk_20_date = df_hkk_20.groupby('Date')['Ex Volume'].sum().reset_index()
df_nothkk_20_date = df_nothkk_20.groupby(
    'Date')['Ex Volume'].sum().reset_index()

####  trips between cities in Hokkaido

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='hkk_20',
           x=df_hkk_20_date['Date'], y=df_hkk_20_date['Ex Volume'])
])
fig.show()

#### trips from outside hokkaido 

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='nothkk_20',
           x=df_nothkk_20_date['Date'], y=df_nothkk_20_date['Ex Volume'])
])
fig.show()

## analysis on trips change between cities in hokkaido

In [ ]:
## top 20 cities, descending by Destination City's volume
list_topcity=df_hkk_20.groupby('Destination City')['Ex Volume'].sum().reset_index().sort_values('Ex Volume',ascending=False).reset_index(drop=True).iloc[:20,0].tolist()

In [ ]:
## data 2020.2.28 and data 2020.5.6

df_hkk_20_228=df_hkk_20[df_hkk_20['Date']=='2020-02-28']

df_hkk_20_506=df_hkk_20[df_hkk_20['Date']=='2020-05-06']

In [ ]:
def data_date(data):
    data=data[['Origin City','Destination City','Ex Volume']]
    data_temp=data[(data['Destination City'].isin(list_topcity)) & (data['Origin City'].isin(list_topcity))]
    data_date=data_temp.groupby(['Origin City','Destination City'])['Ex Volume'].sum().unstack().fillna(0).astype(int)
    return data_date

#### make heatmap figure


In [ ]:

fig = make_subplots(rows=2, cols=1,shared_yaxes=True,shared_xaxes=True,subplot_titles=('February 28, 2020','May 6, 2020'),vertical_spacing=0.1)
 
z1=data_date(df_hkk_20_228).values
z2=data_date(df_hkk_20_506).values
x1=data_date(df_hkk_20_228).columns
x2=data_date(df_hkk_20_506).columns
y1=data_date(df_hkk_20_228).index
y2=data_date(df_hkk_20_506).index


fig1 = ff.create_annotated_heatmap(x=x1.tolist(), y=y1.tolist(), z=z1, hoverinfo='z',colorscale='oryel')
fig2 = ff.create_annotated_heatmap(x=x2.tolist(), y=y2.tolist(), z=z2, hoverinfo='z',colorscale='oryel')

fig.add_trace(fig1.data[0], 1, 1)
fig.add_trace(fig2.data[0], 2, 1)

for i in range(len(fig1.layout.annotations)):
    fig1.layout.annotations[i].font.size = 8
for i in range(len(fig2.layout.annotations)):
    fig2.layout.annotations[i].font.size = 8

annot1 = list(fig1.layout.annotations)
annot2 = list(fig2.layout.annotations)
   
for k in range(len(annot2)):
    annot2[k]['xref'] = 'x2'
    annot2[k]['yref'] = 'y2'

fig.update_layout(annotations=annot1+annot2)
   

fig.update_layout(
    font=dict(
        family="Arial",
        size=11,
        color="#555555"
    )
)


fig.update_layout(width = 1000, height = 1000)


##   analysis on trips change on desitinations

In [18]:
df_hkk_19 = df_drop_19[df_drop_19['Origin Prefecture'] == '北海道']
df_nothkk_19 = df_drop_19[df_drop_19['Origin Prefecture'] != '北海道']

df_hkk_20 = df_drop_20[df_drop_20['Origin Prefecture'] == '北海道']
df_nothkk_20 = df_drop_20[df_drop_20['Origin Prefecture'] != '北海道']

#### data nomalization

In [19]:
total_vol_19_hkk = df_hkk_19['Ex Volume'].sum()
total_vol_20_hkk = df_hkk_20['Ex Volume'].sum()

total_vol_19_nothkk = df_nothkk_19['Ex Volume'].sum()
total_vol_20_nothkk = df_nothkk_20['Ex Volume'].sum()

In [20]:
df_hkk_19.loc[:, 'rela_vol_19_hkk'] = df_hkk_19['Ex Volume']/total_vol_19_hkk
df_nothkk_19.loc[:, 'rela_vol_19_nothkk'] = df_nothkk_19['Ex Volume'] / \
    total_vol_19_nothkk

df_hkk_20.loc[:, 'rela_vol_20_hkk'] = df_hkk_20['Ex Volume']/total_vol_20_hkk
df_nothkk_20.loc[:, 'rela_vol_20_nothkk'] = df_nothkk_20['Ex Volume'] / \
    total_vol_20_nothkk

#### caculate the volume of detination

In [ ]:
df_20_Dhkk = df_hkk_20.groupby('Destination City')['rela_vol_20_hkk'].sum(
).reset_index().sort_values('rela_vol_20_hkk', ascending=False)

df_20_Dnothkk = df_nothkk_20.groupby('Destination City')['rela_vol_20_nothkk'].sum(
).reset_index().sort_values('rela_vol_20_nothkk', ascending=False)

In [ ]:
df_19_Dhkk = df_hkk_19.groupby('Destination City')['rela_vol_19_hkk'].sum(
).reset_index().sort_values('rela_vol_19_hkk', ascending=False)

df_19_Dnothkk = df_nothkk_19.groupby('Destination City')['rela_vol_19_nothkk'].sum(
).reset_index().sort_values('rela_vol_19_nothkk', ascending=False)

In [ ]:
df_merge_19 = pd.merge(df_19_Dhkk, df_19_Dnothkk,
                       on='Destination City', how='outer')

df_merge_20 = pd.merge(df_20_Dhkk, df_20_Dnothkk,
                       on='Destination City', how='outer')

In [ ]:
df_merge = pd.merge(df_merge_19, df_merge_20,
                    on='Destination City', how='outer')

#### difference caculation

In [ ]:
df_merge['diff_hkk'] = df_merge_20['rela_vol_20_hkk'] - \
    df_merge_19['rela_vol_19_hkk']
df_merge['diff_nothkk'] = df_merge_20['rela_vol_20_nothkk'] - \
    df_merge_19['rela_vol_19_nothkk']

In [ ]:
df_merge = df_merge.sort_values('diff_nothkk', ascending=True)

In [ ]:
df_merge['diff_hkk'] = df_merge['diff_hkk']*100
df_merge['diff_nothkk'] = df_merge['diff_nothkk']*100

#### make bar chart 

In [ ]:
destination_city = df_merge[:30].loc[:, 'Destination City'].tolist()
fig = go.Figure(data=[
    go.Bar(name='other prefectures', x=destination_city,
           y=df_merge[:30].loc[:, 'diff_nothkk'].tolist(), marker_color=['#e16428']*35),
    go.Bar(name='Hokkaido', x=destination_city,
           y=df_merge[:30].loc[:, 'diff_hkk'].tolist(), marker_color=['#bebec0']*35)
])


fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial', size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=True,
        showline=False,
        showticklabels=True,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)

fig.update_layout(legend=dict(
    x=0.75,
    y=1.3,
),
    barmode='relative', xaxis_tickangle=-45)

fig.update_layout(
    title='The Most Affected City in Hokkaido',

    xaxis_tickfont_size=10,
    yaxis=dict(
        title='Human Mobility Inflow Changes(%)',
        color='rgb(37,37,37)',
        titlefont_size=16,
        tickfont_size=10,
    ),
)

## analysis on trips change from outside hokkaido

In [ ]:
df_hkk_19 = df_drop_19[df_drop_19['Origin Prefecture'] == '北海道']
df_nothkk_19 = df_drop_19[df_drop_19['Origin Prefecture'] != '北海道']

In [ ]:
df_hkk_19_date = df_hkk_19.groupby('date')['Ex Volume'].sum(
).reset_index().rename(columns={'Ex Volume': 'vol_19_hkk'})
df_nothkk_19_date = df_nothkk_19.groupby('date')['Ex Volume'].sum(
).reset_index().rename(columns={'Ex Volume': 'vol_19_nothkk'})
df_hkk_20_date = df_hkk_20.groupby('date')['Ex Volume'].sum(
).reset_index().rename(columns={'Ex Volume': 'vol_20_hkk'})
df_nothkk_20_date = df_nothkk_20.groupby('date')['Ex Volume'].sum(
).reset_index().rename(columns={'Ex Volume': 'vol_20_nothkk'})

In [ ]:
merge_19_date = pd.merge(df_hkk_19_date, df_nothkk_19_date, on='date')
merge_20_date = pd.merge(df_hkk_20_date, df_nothkk_20_date, on='date')

In [ ]:
merge_date = pd.merge(merge_19_date, merge_20_date, on='date')

In [ ]:
array_data_x = merge_date.values.T[0]

#### 7-day average

In [ ]:
array_temp_1 = pd.DataFrame(merge_date.iloc[0]).T
array_temp_2 = pd.DataFrame(merge_date.iloc[-1]).T
merge_date = merge_date.append([array_temp_1]*3)
merge_date = merge_date.append([array_temp_2]*3)

In [ ]:
data_temp = merge_date.sort_values(
    'date', ascending=True).iloc[:, 1:].astype(int)

In [ ]:
array_data_y = data_temp.values.T

In [ ]:
list_1 = array_data_y[0].tolist()
N = 7
cumsum_1, moving_aves_1 = [0], []

for i, x in enumerate(list_1, 1):

    cumsum_1.append(cumsum_1[i-1] + x)
    if i >= N:
        moving_ave_1 = (cumsum_1[i] - cumsum_1[i-N])/N

        moving_aves_1.append(moving_ave_1)

In [ ]:
list_2 = array_data_y[1].tolist()
N = 7
cumsum_2, moving_aves_2 = [0], []

for i, x in enumerate(list_2, 1):

    cumsum_2.append(cumsum_2[i-1] + x)
    if i >= N:
        moving_ave_2 = (cumsum_2[i] - cumsum_2[i-N])/N

        moving_aves_2.append(moving_ave_2)

In [ ]:
list_3 = array_data_y[2].tolist()
N = 7
cumsum_3, moving_aves_3 = [0], []

for i, x in enumerate(list_3, 1):

    cumsum_3.append(cumsum_3[i-1] + x)
    if i >= N:
        moving_ave_3 = (cumsum_3[i] - cumsum_3[i-N])/N

        moving_aves_3.append(moving_ave_3)

In [ ]:
list_4 = array_data_y[3].tolist()
N = 7
cumsum_4, moving_aves_4 = [0], []

for i, x in enumerate(list_4, 1):

    cumsum_4.append(cumsum_4[i-1] + x)
    if i >= N:
        moving_ave_4 = (cumsum_4[i] - cumsum_4[i-N])/N

        moving_aves_4.append(moving_ave_4)

#### make line plot

In [ ]:
title = 'Human Mobility Inflow Changes on Time Series'
labels = ['2019', '2020']
colors = ['#bebec0', '#008cc3']

x_data = np.vstack((array_data_x,)*2)
y_data = np.array([moving_aves_3, moving_aves_4])

fig = go.Figure()

for i in range(0, 2):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
                             name=labels[i],
                             line=dict(color=colors[i], width=2),
                             connectgaps=True,
                             ))
    fig.add_trace(go.Scatter(
        x=[x_data[0][-1], x_data[1][-1]],
        y=[y_data[0][-1], y_data[1][-1]],
        mode='markers',
        marker=dict(color=['#bebec0', '#008cc3', '#bebec0', '#008cc3'], size=6)
    ))

    fig.add_trace(go.Scatter(x=[x_data[1][27], x_data[1][27]], y=[0, 32746],
                             mode='lines',
                             name='event date',
                             line=dict(color='#e16428',
                                       width=1,
                                       dash='dot')))
    fig.add_trace(go.Scatter(x=[x_data[1][46], x_data[1][46]], y=[0, 32746],
                             mode='lines',
                             name='event date',
                             line=dict(color='#e16428',
                                       width=1,
                                       dash='dot')))
    fig.add_trace(go.Scatter(x=[x_data[1][73], x_data[1][73]], y=[0, 32746],
                             mode='lines',
                             name='event date',
                             line=dict(color='#e16428',
                                       width=1,
                                       dash='dot')))
    fig.add_trace(go.Scatter(x=[x_data[1][121], x_data[1][121]], y=[0, 32746],
                             mode='lines',
                             name='event date',
                             line=dict(color='#e16428',
                                       width=1,
                                       dash='dot')))

    fig.update_layout(
        xaxis=dict(
            showline=True,
            showgrid=False,
            showticklabels=True,
            linecolor='rgb(204, 204, 204)',
            linewidth=2,
            ticks='outside',
            tickfont=dict(
                family='Arial', size=12,
                color='rgb(82, 82, 82)',
            ),
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=True,
            showline=True,
            showticklabels=False,
        ),
        autosize=False,
        margin=dict(
            autoexpand=False,
            l=100,
            r=20,
            t=110,
        ),
        showlegend=False,
        plot_bgcolor='white'
    )

    annotations = []

    for y_trace, y_trace, label, color in zip(y_data, y_data, labels, colors):

        annotations.append(dict(xref='paper', x=0.95, y=y_data[0][-1],
                                xanchor='left', yanchor='middle',
                                text=labels[0],
                                font=dict(family='Arial',
                                          size=14,
                                          color=colors[0]),
                                showarrow=False))
        annotations.append(dict(xref='paper', x=0.95, y=y_data[1][-1],
                                xanchor='left', yanchor='middle',
                                text=labels[1],
                                font=dict(family='Arial',
                                          size=14,
                                          color=colors[1]),
                                showarrow=False))

        annotations.append(dict(x=x_data[1][27], y=36000,
                                xanchor='right', yanchor='top',
                                text='lockdown was <br> declared',
                                font=dict(family='Arial',
                                          size=12,
                                          color='#e16428'),
                                showarrow=False))
        annotations.append(dict(x=x_data[1][121], y=39000,
                                xanchor='left', yanchor='top',
                                text='second time<br>lockdown was lifted',
                                font=dict(family='Arial',
                                          size=12,
                                          color='#e16428'),
                                showarrow=False))
        annotations.append(dict(x=x_data[1][73], y=36000,
                                xanchor='left', yanchor='top',
                                text='second time lockdown',
                                font=dict(family='Arial',
                                          size=12,
                                          color='#e16428'),
                                showarrow=False))
        annotations.append(dict(x=x_data[1][46], y=36000,
                                xanchor='center', yanchor='top',
                                text='lockdown was lift',
                                font=dict(family='Arial',
                                          size=12,
                                          color='#e16428'),
                                showarrow=False))

        annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.1,
                                xanchor='left', yanchor='bottom',
                                text='Human Mobility Inflow Changes on Time Series',
                                font=dict(family='Arial',
                                          size=16,
                                          color='rgb(37,37,37)'),
                                showarrow=False))

fig.update_layout(annotations=annotations, xaxis_tickangle=-45)


fig.show()